In [20]:
import numpy as np

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

from crepes import WrapRegressor
from crepes.extras import DifficultyEstimator, MondrianCategorizer

import matplotlib.pyplot as plt

### **Cargar datos**

In [10]:
dataset = fetch_california_housing()

X = dataset.data.astype(float)
y = dataset.target.astype(float)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

X_test, X_cal, y_test, y_cal = train_test_split(X_test, y_test, test_size=0.5)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) 
X_test = scaler.transform(X_test)
X_cal = scaler.transform(X_cal)  

print(X_train.size)
print(X_test.size)
print(X_cal.size)

115584
24768
24768


### **Entrenar modelo**

In [11]:

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=0)

predictions = model.predict(X_test).flatten()

/home/antonio/code/uncertainty-pinns/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


97/97 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


### **Keras 2 sk-learn**

In [17]:
from scikeras.wrappers import KerasRegressor

model_sklearn = KerasRegressor(model=model, optimizer="adam", epochs=20, batch_size=32, validation_split=0.2, verbose=0)

### **Calibración**

In [19]:
nn = WrapRegressor(model_sklearn)

nn.fit(X_train, y_train)
nn.calibrate(X_cal, y_cal)

display(nn)

WrapRegressor(learner=KerasRegressor(
	model=<Sequential name=sequential_1, built=True>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=None
	metrics=None
	batch_size=32
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.2
	shuffle=True
	run_eagerly=False
	epochs=20
), calibrated=True, predictor=ConformalRegressor(fitted=True, normalized=False, mondrian=False))

### **Predicción** (intervalos de amplitud fija)

In [14]:
intervals = nn.predict_int(X_test, confidence=0.99)

display(intervals)

array([[-0.9661181 ,  3.11808515],
       [ 1.25700903,  5.34121227],
       [-0.06518471,  4.01901853],
       ...,
       [ 2.22546887,  6.30967212],
       [-1.57047218,  2.51373106],
       [ 0.81511402,  4.89931726]])

### **Predicción** (intervalos variables)

In [21]:
learner_prop = nn.learner

de_var = DifficultyEstimator()

de_var.fit(X=X_train, learner=learner_prop, scaler=True)

display(de_var)

mc = MondrianCategorizer()

mc.fit(X_cal, de=de_var, no_bins=20)

display(mc)

nn_mond = WrapRegressor(learner_prop)

nn_mond.calibrate(X_cal, y_cal, mc=mc)

display(nn_mond)

intervals_mond = nn_mond.predict_int(X_test, y_min=0, y_max=1)

display(intervals_mond)

ValueError: learner is missing the attribute estimators_